<a href="https://colab.research.google.com/github/mayumiluiza/shark_attack/blob/main/Project_1_SharkAttack_limpezadedados_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
#Importando bibliotecas
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns


In [67]:
#Função para abrir arquivo do google sheets
def read_from_gsheets(spreadsheet):
    """
    Transform url into csv 
    """
    working_spreadsheet = spreadsheet.replace('/edit?usp=sharing','/export?format=csv')
    
    return pd.read_csv(working_spreadsheet)

In [68]:
#Abrindo arquivo do google sheets
sharkattack_main = read_from_gsheets('https://docs.google.com/spreadsheets/d/1tFrVsabcTh4J-T6par8RYUFSvPRmVCjj/edit?usp=sharing&ouid=112931314534247376345&rtpof=true&sd=true')

In [69]:
#Limpando colunas e linhas extras que não são originárias da planilha oficial
sharkattack_main = sharkattack_main.iloc[:6840, :15]
sharkattack_main

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,25-Mar-2023,2023,Unprovoked,USA,Florida,"Blue Fish Channel, Key West, Monroe County",Kite-Surfing,Kevin Carlton Scott,M,67,Lacertions to right calf,N,14h00,NaN,"Yahoo News, 3/28/2023"
1,22-Mar-2023,2023,Unprovoked,AUSTRALIA,Western Australia,Delta Island,Wading,Adam Norton,M,30s,Lower left leg and foot injured,N,18h00,NaN,"A. Currie, GSAF"
2,19-Mar-2023,2023,Unprovoked,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,60,Left hand and left leg.,N,12h30,NaN,"Khon2, 3/19/2023"
3,17-Mar-2023,2023,Unprovoked,MEXICO,Quintana Roo,Cancún,Swimming,Dillon Armijo,M,10,Significant injury to leg,N,NaN,NaN,"Denver7, 3/29/2023"
4,16-Mar-2023,2023,Unprovoked,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,NaN,NaN,N,08h00,5m to 6m shark,"B. Myatt, GSAF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6835,Before 1903,0000,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6836,Before 1903,0000,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6837,1900-1905,0000,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6838,1883-1889,0000,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [70]:
#Hipóteses para Validar
# Hipótese mãe: Existem diferença entre quantidade de ataques dependendo do sexo, país e idade
##1 Ataques de tubarão acontecem 2x mais nos EUA do que eu qualquer outro lugar do mundo
##2 Ataques de tubarão acontecem mais em homens do que em mulheres 
##3 Ataques de tubarão acontecem mais em adultos (+18 anos) do que em crianças.

In [71]:
sharkattack_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6840 entries, 0 to 6839
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    6840 non-null   object
 1   Year                    6838 non-null   object
 2   Type                    6820 non-null   object
 3   Country                 6790 non-null   object
 4   Area                    6362 non-null   object
 5   Location                6279 non-null   object
 6   Activity                6257 non-null   object
 7   Name                    6621 non-null   object
 8   Sex                     6263 non-null   object
 9   Age                     3869 non-null   object
 10  Injury                  6805 non-null   object
 11  Fatal (Y/N)             6282 non-null   object
 12  Time                    3339 non-null   object
 13  Species                 3757 non-null   object
 14  Investigator or Source  6821 non-null   object
dtypes: o

# Coluna Country

In [72]:
#COLUNA COUNTRY - Alterando valores NaN (part1.1: Descobrindo quantas linhas são NaN)
sharkattack_main.loc[sharkattack_main['Country'].isnull()]
len(sharkattack_main.loc[sharkattack_main['Country'].isnull()])

50

In [73]:
#COLUNA COUNTRY - Alterando valores NaN (part1.2: Descobrindo quantas linhas são NaN)
sharkattack_main.loc[sharkattack_main['Country'].isnull()]


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
572,Reported 13-Nov-2017,2017,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,M,24,Puncture wounds to feet,N,NaN,NaN,Instagram
1038,Aug-2014,2014,Invalid,NaN,NaN,NaN,Sea disaster,Cuban refugees,M,NaN,Shark involvement prior to death not confirmed,NaN,NaN,Shark involvement not confirmed,"Associated Press, 11/27/2014"
3474,Ca. 1983,1983,Unprovoked,NaN,English Channel,NaN,Swimming,Padma Shri Taranath Narayan Shenoy,M,NaN,Left leg bitten,N,NaN,NaN,"Times of India, 2/5/2012"
3904,Nov-1970,1970,Unprovoked,NaN,NaN,NaN,NaN,Heinz Plotsky,M,NaN,Extensive injuries,N,NaN,NaN,"H.D. Baldridge (1994), SAF Case #1645"
3913,02-Aug-1970,1970,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,NaN,"Sharks scavenged on bodies, but no record of t...",NaN,Afternoon,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970"
3914,5-Jul-1970,1970,Unprovoked,NaN,NaN,NaN,NaN,male,M,NaN,Finger or toe severed,N,Night,Mako shark,"H.D. Baldridge (1994), SAF Case #1628"
3921,Apr-1970,1970,Provoked,NaN,NaN,NaN,Freediving,Lionel Jarvis,M,NaN,Arm abraded & lacerated. Recorded as PROVOKED ...,N,NaN,Wobbegong shark,"H.D. Baldridge (1994), SAF Case #1616"
3925,5-Feb-1970,1970,Unprovoked,NaN,NaN,NaN,Wading,Sally Anne Irvine,F,8,Lacerations to lower leg,N,NaN,Carpet shark,H.D. Baldridge (1994) SAF Case #1626
3951,Aug-1969,1969,Unprovoked,NaN,NaN,NaN,NaN,Rodney Hughes,M,25,Am lacerated,N,NaN,NaN,H.D. Baldridge (1994) SAF Case #1602
4131,21-Oct-1965,1965,Unprovoked,NaN,NaN,Florida Strait,The boat Caribou II sank,Mario Castellanos,M,39,Survived,N,NaN,NaN,"Lodi News Sentinel, 10/30/1965"


In [74]:
#COLUNA COUNTRY - Alterando valores NaN (part2: Dropando valores que são impossíveis completar o país)
index_countries_nan = sharkattack_main.loc[(sharkattack_main['Country'].isnull()) & (sharkattack_main['Location'].isnull()) & (sharkattack_main['Area'].isnull())].index
sharkattack_main = sharkattack_main.drop(index_countries_nan,axis=0).reset_index()


In [75]:
sharkattack_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6811 entries, 0 to 6810
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   6811 non-null   int64 
 1   Date                    6811 non-null   object
 2   Year                    6809 non-null   object
 3   Type                    6791 non-null   object
 4   Country                 6790 non-null   object
 5   Area                    6362 non-null   object
 6   Location                6279 non-null   object
 7   Activity                6235 non-null   object
 8   Name                    6592 non-null   object
 9   Sex                     6236 non-null   object
 10  Age                     3863 non-null   object
 11  Injury                  6778 non-null   object
 12  Fatal (Y/N)             6256 non-null   object
 13  Time                    3335 non-null   object
 14  Species                 3749 non-null   object
 15  Inve

In [76]:
#COLUNA COUNTRY - Alterando valores NaN (part3: Alterando valores que são possíveis completar o país)
sharkattack_main.loc[(sharkattack_main['Country'].isnull()) & (~sharkattack_main['Location'].isnull()|~sharkattack_main['Area'].isnull())]


,index,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
3472,3474,Ca. 1983,1983,Unprovoked,NaN,English Channel,NaN,Swimming,Padma Shri Taranath Narayan Shenoy,M,NaN,Left leg bitten,N,NaN,NaN,"Times of India, 2/5/2012"
3910,3913,02-Aug-1970,1970,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,NaN,NaN,NaN,"Sharks scavenged on bodies, but no record of t...",NaN,Afternoon,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970"
4124,4131,21-Oct-1965,1965,Unprovoked,NaN,NaN,Florida Strait,The boat Caribou II sank,Mario Castellanos,M,39,Survived,N,NaN,NaN,"Lodi News Sentinel, 10/30/1965"
4536,4544,26-Jan-1960,1960,Sea Disaster,NaN,"Between Timor & Darwin, Australia",NaN,Portuguese Airliner with 9 people aboard went ...,NaN,NaN,NaN,"As searchers approached wreckage, sharks circl...",N,NaN,NaN,"V.M. Coppleson (1962), p.260"
4749,4757,13-Sep-1956,1956,Unprovoked,NaN,Near the Andaman & Nicobar Islands,NaN,Climbing back on ship,male,M,NaN,FATAL,Y,P.M.,Blue shark,M. Hosina
4784,4792,1956,1956,Sea Disaster,NaN,Between Comores & Madagascar,Geyser Bank,Shipwreck,"Captain Eric Hunt, the cook & a French passenger",M,NaN,FATAL,Y,NaN,NaN,dinofish.com
5019,5027,Dec-1949,1949,Sea Disaster,NaN,Caribbean Sea,Between Cuba & Costa Rica,"Sea Disaster, sinking of the motorship Wingate","Albert Battles, James Dean & 4 crew",M,NaN,Fatal or drowning or scavenging,Y,NaN,Shark involvement not confirmed,"Canberra Times, 1/6/1950"
5160,5168,24-Oct-1944,1944,Sea Disaster,NaN,NaN,225 miles east of Hong Kong,Japanese POW ship Arisan Maru with 1800 Americ...,NaN,M,NaN,Most of the men drowned & some were taken by s...,Y,>17h30,NaN,internet (multiple)
5221,5229,Nov-1942,1942,Sea Disaster,NaN,Off South American coast,NaN,Dutch merchant ship Zaandam torpedoed by the ...,NaN,M,NaN,FATAL,Y,NaN,NaN,"M. Murphy; V.M. Coppleson (1962), pp.207-208"
5233,5241,Jun-1942,1942,Unprovoked,NaN,300 miles east of St. Thomas (Virgin Islands),NaN,On life raft tethered to lifeboat. A seaman pu...,male,M,NaN,Forearm lacerated,N,NaN,NaN,"V.M. Coppleson (1962), p.258"


In [77]:
#COLUNA COUNTRY - Alterando valores NaN (part3.1: Alterando valores que são possíveis completar o país)
sharkattack_main.loc[sharkattack_main["Area"] == "St Helena", "Country"] = "ST HELENA, BRITISH OVERSEAS TERRITORY"
sharkattack_main.loc[sharkattack_main["Location"] == "Île Saint-Paul", "Country"] = "ÎLE SAINT-PAUL, FRENCH OVERSEAS TERRITORY"

In [78]:
#COLUNA COUNTRY - Ajustando valores(part 3.2: Alterando alguns países)
sharkattack_main.loc[sharkattack_main["Country"] == "JA MAICA", "Country"] = "JAMAICA"
sharkattack_main.loc[sharkattack_main["Country"] == "OKINAWA", "Country"] = "JAPAN"
sharkattack_main.loc[sharkattack_main["Country"] == "UNITED ARAB EMIRATES (UAE)", "Country"] = "UNITED ARAB EMIRATES"
sharkattack_main.loc[sharkattack_main["Country"] == "ENGLAND", "Country"] = "UNITED KINGDOM"
sharkattack_main.loc[sharkattack_main["Country"] == "SCOTLAND", "Country"] = "UNITED KINGDOM"

In [79]:
#COLUNA COUNTRY - Alterando valores NaN (part3.2: Alterando valores que não são possíveis completar o país)
sharkattack_main["Update Countries"] = sharkattack_main['Country'].map(lambda country: "IN BETWEEN COUNTRIES OR HIGH SEA" if country != country else country)
sharkattack_main = sharkattack_main.loc[:,['index', 'Date', 'Year', 'Type', 'Country', 'Update Countries', 'Area', 'Location','Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source']]
sharkattack_main

,index,Date,Year,Type,Country,Update Countries,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,0,25-Mar-2023,2023,Unprovoked,USA,USA,Florida,"Blue Fish Channel, Key West, Monroe County",Kite-Surfing,Kevin Carlton Scott,M,67,Lacertions to right calf,N,14h00,NaN,"Yahoo News, 3/28/2023"
1,1,22-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Delta Island,Wading,Adam Norton,M,30s,Lower left leg and foot injured,N,18h00,NaN,"A. Currie, GSAF"
2,2,19-Mar-2023,2023,Unprovoked,USA,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,60,Left hand and left leg.,N,12h30,NaN,"Khon2, 3/19/2023"
3,3,17-Mar-2023,2023,Unprovoked,MEXICO,MEXICO,Quintana Roo,Cancún,Swimming,Dillon Armijo,M,10,Significant injury to leg,N,NaN,NaN,"Denver7, 3/29/2023"
4,4,16-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,NaN,NaN,N,08h00,5m to 6m shark,"B. Myatt, GSAF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6806,6835,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6807,6836,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6808,6837,1900-1905,0000,Unprovoked,USA,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6809,6838,1883-1889,0000,Unprovoked,PANAMA,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [80]:
#COLUNA COUNTRY - Ajustando valores (part1: Analisando os valores únicos de Country)
len(list(sharkattack_main['Update Countries'].unique()))

219

In [81]:
#COLUNA COUNTRY - Ajustando valores (part2: Deixando todos os valores em um mesmo padrão)
sharkattack_main['Update Countries'] = sharkattack_main['Update Countries'].str.upper()
sharkattack_main['Update Countries'] = sharkattack_main['Update Countries'].str.strip()
sharkattack_main['Update Countries'] = sharkattack_main['Update Countries'].str.replace(r'\?$',"", regex=True)

In [82]:
#COLUNA COUNTRY - Ajustando valores  (part3: definindo quais são os países únicos)
shark_countries = list(sharkattack_main['Update Countries'].unique())

In [83]:
#COLUNA COUNTRY - Ajustando valores  (part4: analisando se todos países estão com grafias certas)
countries_list = read_from_gsheets('https://docs.google.com/spreadsheets/d/1m4jcr2736SL20HbEVeWPkAap1FVOAvGwF5bkgrAO8vs/edit?usp=sharing')
countries_list = list(countries_list["Update Countries"])

unique_countries = list(set(shark_countries) - set(countries_list))
unique_countries

['COLUMBIA',
 'SOUTH ATLANTIC OCEAN',
 'CEYLON (SRI LANKA)',
 'BRITISH ISLES',
 'TURKS & CAICOS',
 'AFRICA',
 'OCEAN',
 'ASIA',
 'NEVIS',
 'MEDITERRANEAN SEA',
 'CEYLON',
 'THE BALKANS',
 'ST HELENA, BRITISH OVERSEAS TERRITORY',
 'RED SEA',
 'SAN DOMINGO',
 'BRITISH WEST INDIES',
 'ANDAMAN / NICOBAR ISLANDAS',
 'TRINIDAD & TOBAGO',
 'REUNION ISLAND',
 'NEW BRITAIN',
 'ANTIGUA',
 'SOUTH PACIFIC OCEAN',
 'PACIFIC OCEAN',
 'PERSIAN GULF',
 'COAST OF AFRICA',
 'SOLOMON ISLANDS / VANUATU',
 'CENTRAL PACIFIC',
 'ADMIRALTY ISLANDS',
 'AZORES',
 'JAVA',
 'MALDIVE ISLANDS',
 'KOREA',
 'NORTH SEA',
 'ST KITTS / NEVIS',
 'FEDERATED STATES OF MICRONESIA',
 'GULF OF ADEN',
 'TOBAGO',
 'INDIAN OCEAN',
 'SOUTH CHINA SEA',
 'EGYPT / ISRAEL',
 'BAY OF BENGAL',
 'ST MARTIN',
 'RED SEA / INDIAN OCEAN',
 'JOHNSTON ISLAND',
 'NEW GUINEA',
 'DIEGO GARCIA',
 'NORTH ATLANTIC OCEAN',
 'NORTHERN ARABIAN SEA',
 'SOUTHWEST PACIFIC OCEAN',
 'BAHREIN',
 'BURMA',
 'CRETE',
 'ST. MARTIN',
 'ATLANTIC OCEAN',
 'ROATAN'

In [84]:
#COLUNA COUNTRY - Ajustando valores(part 5: Descobrindo se há uma coluna com "sea" ou "ocean" ou "river" ou "between countries (/)" ou "gulf ou "Coast"
ocean_df = sharkattack_main.loc[sharkattack_main['Update Countries'].str.contains(r'SEA|OCEAN|RIVER|GULF|BETWEEN|COAST|/|CHANNEL|PACIFIC'),"Update Countries"]

sharkattack_main.loc[sharkattack_main['Update Countries'].isin(ocean_df), 'Update Countries'] = "IN BETWEEN COUNTRIES OR HIGH SEA"

len(sharkattack_main['Update Countries'].unique())

167

In [85]:
#COLUNA COUNTRY - Ajustando valores(part 6: Analisando os novos valores únicos)

sharkattack_main['Update Countries'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'SEYCHELLES',
       'NEW CALEDONIA', 'ARGENTINA', 'FIJI', 'SOUTH AFRICA', 'BAHAMAS',
       'UNITED KINGDOM', 'JAPAN', 'INDONESIA', 'EGYPT', 'JAMAICA',
       'BELIZE', 'MALDIVES', 'FRENCH POLYNESIA', 'THAILAND', 'COLUMBIA',
       'NEW ZEALAND', 'COSTA RICA', 'IN BETWEEN COUNTRIES OR HIGH SEA',
       'CANADA', 'ECUADOR', 'JORDAN', 'ST MARTIN', 'SPAIN',
       'PAPUA NEW GUINEA', 'REUNION ISLAND', 'ISRAEL', 'CHINA', 'SAMOA',
       'IRELAND', 'ITALY', 'COLOMBIA', 'MALAYSIA', 'LIBYA', 'CUBA',
       'MAURITIUS', 'SOLOMON ISLANDS', 'COMOROS', 'REUNION',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'CAPE VERDE',
       'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE',
       'PUERTO RICO', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'TRINIDAD & TOBAGO', 'KIRIBATI', 'DIEGO GARCIA', 'TAIWAN',
       'PALESTINIAN TERRITORIES', 'GUAM', 'NIGERIA', 'TONGA', 'CROATIA',
       'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
      

In [86]:
#COLUNA COUNTRY - Removendo valores duplicados (part 1: Ataques se repetem devido a numero de vitimas)
sharkattack_main.drop_duplicates(subset='index')

,index,Date,Year,Type,Country,Update Countries,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,0,25-Mar-2023,2023,Unprovoked,USA,USA,Florida,"Blue Fish Channel, Key West, Monroe County",Kite-Surfing,Kevin Carlton Scott,M,67,Lacertions to right calf,N,14h00,NaN,"Yahoo News, 3/28/2023"
1,1,22-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Delta Island,Wading,Adam Norton,M,30s,Lower left leg and foot injured,N,18h00,NaN,"A. Currie, GSAF"
2,2,19-Mar-2023,2023,Unprovoked,USA,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,60,Left hand and left leg.,N,12h30,NaN,"Khon2, 3/19/2023"
3,3,17-Mar-2023,2023,Unprovoked,MEXICO,MEXICO,Quintana Roo,Cancún,Swimming,Dillon Armijo,M,10,Significant injury to leg,N,NaN,NaN,"Denver7, 3/29/2023"
4,4,16-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,NaN,NaN,N,08h00,5m to 6m shark,"B. Myatt, GSAF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6806,6835,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6807,6836,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6808,6837,1900-1905,0000,Unprovoked,USA,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6809,6838,1883-1889,0000,Unprovoked,PANAMA,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


In [87]:
sharkattack_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6811 entries, 0 to 6810
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   6811 non-null   int64 
 1   Date                    6811 non-null   object
 2   Year                    6809 non-null   object
 3   Type                    6791 non-null   object
 4   Country                 6792 non-null   object
 5   Update Countries        6811 non-null   object
 6   Area                    6362 non-null   object
 7   Location                6279 non-null   object
 8   Activity                6235 non-null   object
 9   Name                    6592 non-null   object
 10  Sex                     6236 non-null   object
 11  Age                     3863 non-null   object
 12  Injury                  6778 non-null   object
 13  Fatal (Y/N)             6256 non-null   object
 14  Time                    3335 non-null   object
 15  Spec

# Coluna Age

In [88]:
#COLUNA AGE - Investigando valores (part 1: Descobrindo os valores)
sharkattack_main['Age'].unique()

array(['67', '30s', '60', '10', nan, '15', '14', '6', '32', '59', '41',
       '16', '8', '49', '34', '53', '30', '26', '68', '43', '50', '29',
       '40', '51', '19', '38', '17', '31', '39', '58', '13', '33', '28',
       '40s', '35', '62', 'teen', '20', '22', '56', 'Teen', '12', '50s',
       '21', '42', '36', '18', '37', 'M', '9', '24', '11', '27', '57',
       '25', '!6', '64', '!!', '47', '55', '7', '71', '48', '54', '75',
       '45 and 15', '46', '61', '73', '52', '70', '23', '4', '63', '45',
       '44', '28 & 22', '22, 57, 31', '60s', "20's", '65', '74', '9 & 60',
       '20s', 'a minor', '69', '3', '82', '66', '72', '18 months', '77',
       '28 & 26', '5', '86', '18 or 20', '12 or 13', '46 & 34',
       '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84', '\xa0 ', ' ',
       '30 or 36', '6½', '21 & ?', '33 or 37', 'mid-30s', '23 & 20',
       ' 30', '7      &    31', ' 28', '20?', "60's", '32 & 30',
       '16 to 18', '87', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ',
    

In [89]:
#COLUNA AGE - Criando coluna auxiliar (part 1: criando coluna New Age)
sharkattack_main['New Age'] = sharkattack_main['Age']

sharkattack_main = sharkattack_main.loc[:,['index', 'Date', 'Year', 'Type', 'Country', 'Update Countries', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'New Age', 'Injury', 'Fatal (Y/N)',
       'Time', 'Species ', 'Investigator or Source']]

In [90]:
#COLUNA AGE - Ajustando NaN possíveis (part 1: ajustando valores com valores ficticios a partir da coluna Name)
regex_kid= r'\bchild\b|\bchildren\b|\bteen\b|\bteenager\b|\byouth\b'
sharkattack_main.loc[sharkattack_main['Name'].str.contains(regex_kid, case=False, na=False), "New Age"] = "10"

In [91]:
#COLUNA AGE - Limpando as strings (part 1: transformando valores de crianças)
sharkattack_main.loc[sharkattack_main["New Age"] == "teen", "New Age"] = "10"
sharkattack_main.loc[sharkattack_main["New Age"] == "Teen", "New Age"] = "10"
sharkattack_main.loc[sharkattack_main["New Age"] == "Teens", "New Age"] = "10"
sharkattack_main.loc[sharkattack_main["New Age"] == "!6", "New Age"] = "16"
sharkattack_main.loc[sharkattack_main["New Age"] == "45 and 15", "New Age"] = "15"
sharkattack_main.loc[sharkattack_main["New Age"] == "9 & 60", "New Age"] = "9"
sharkattack_main.loc[sharkattack_main["New Age"] == "12 or 13", "New Age"] = "12"
sharkattack_main.loc[sharkattack_main["New Age"] == "8 or 10", "New Age"] = "8"
sharkattack_main.loc[sharkattack_main["New Age"] == "6½", "New Age"] = "6"
sharkattack_main.loc[sharkattack_main["New Age"] == "7      &    31", "New Age"] = "7"
sharkattack_main.loc[sharkattack_main["New Age"] == "16 to 18", "New Age"] = "16"
sharkattack_main.loc[sharkattack_main["New Age"] == "9 & 12", "New Age"] = "9"
sharkattack_main.loc[sharkattack_main["New Age"] == "9 months", "New Age"] = "1"
sharkattack_main.loc[sharkattack_main["New Age"] == "a minor", "New Age"] = "17"
sharkattack_main.loc[sharkattack_main["New Age"] == "18 months", "New Age"] = "1"
sharkattack_main.loc[sharkattack_main["New Age"] == "9 months", "New Age"] = "1"
sharkattack_main.loc[sharkattack_main["New Age"] == "1", "New Age"] = "1"
sharkattack_main.loc[sharkattack_main["New Age"] == "17 & 35", "New Age"] = "17"
sharkattack_main.loc[sharkattack_main["New Age"] == "13 or 18", "New Age"] = "13"
sharkattack_main.loc[sharkattack_main["New Age"] == "7 or 8", "New Age"] = "7"
sharkattack_main.loc[sharkattack_main["New Age"] == "9 or 10", "New Age"] = "7"
sharkattack_main.loc[sharkattack_main["New Age"] == "2 to 3 months", "New Age"] = "1"
sharkattack_main.loc[sharkattack_main["New Age"] == '"young"', "New Age"] = "17"
sharkattack_main.loc[sharkattack_main["New Age"] == "17 & 16", "New Age"] = "16"
sharkattack_main.loc[sharkattack_main["New Age"] == "Both 11", "New Age"] = "11"
sharkattack_main.loc[sharkattack_main["New Age"] == "young", "New Age"] = "17"
sharkattack_main.loc[sharkattack_main["New Age"] == '?    &   14', "New Age"] = "14"
sharkattack_main.loc[sharkattack_main["New Age"] == '10 or 12', "New Age"] = "10"
sharkattack_main.loc[sharkattack_main["New Age"] == '2½', "New Age"] = "2"
sharkattack_main.loc[sharkattack_main["New Age"] == "13 or 14", "New Age"] = "13"

In [92]:
#COLUNA AGE - Limpando as strings (part 2: transformando valores de adultos)
sharkattack_main.loc[sharkattack_main["New Age"] == "20s", "New Age"] = "20"
sharkattack_main.loc[sharkattack_main["New Age"] == "20's", "New Age"] = "20"
sharkattack_main.loc[sharkattack_main["New Age"] == "30s", "New Age"] = "30"
sharkattack_main.loc[sharkattack_main["New Age"] == "40s", "New Age"] = "40"
sharkattack_main.loc[sharkattack_main["New Age"] == "50s", "New Age"] = "50"
sharkattack_main.loc[sharkattack_main["New Age"] == "60s", "New Age"] = "60"
sharkattack_main.loc[sharkattack_main["New Age"] == "60's", "New Age"] = "60"
sharkattack_main.loc[sharkattack_main["New Age"] == "28 & 22", "New Age"] = "22"
sharkattack_main.loc[sharkattack_main["New Age"] == "22, 57, 31", "New Age"] = "22"
sharkattack_main.loc[sharkattack_main["New Age"] == "28 & 26", "New Age"] = "26"
sharkattack_main.loc[sharkattack_main["New Age"] == "18 or 20", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "18 & 20", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "18 to 22", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "46 & 34", "New Age"] = "34"
sharkattack_main.loc[sharkattack_main["New Age"] == "28, 23 & 30", "New Age"] = "23"
sharkattack_main.loc[sharkattack_main["New Age"] == "36 & 26", "New Age"] = "26"
sharkattack_main.loc[sharkattack_main["New Age"] == "30 or 36", "New Age"] = "30"
sharkattack_main.loc[sharkattack_main["New Age"] == "21 & ?", "New Age"] = "21"
sharkattack_main.loc[sharkattack_main["New Age"] == "33 or 37", "New Age"] = "33"
sharkattack_main.loc[sharkattack_main["New Age"] == "mid-30s", "New Age"] = "35"
sharkattack_main.loc[sharkattack_main["New Age"] == "23 & 20", "New Age"] = "20"
sharkattack_main.loc[sharkattack_main["New Age"] == "20?", "New Age"] = "20"
sharkattack_main.loc[sharkattack_main["New Age"] == "32 & 30", "New Age"] = "30"
sharkattack_main.loc[sharkattack_main["New Age"] == "Elderly", "New Age"] = "60"
sharkattack_main.loc[sharkattack_main["New Age"] == "mid-20s", "New Age"] = "25"
sharkattack_main.loc[sharkattack_main["New Age"] == "21 or 26", "New Age"] = "25"
sharkattack_main.loc[sharkattack_main["New Age"] == ">50", "New Age"] = "51"
sharkattack_main.loc[sharkattack_main["New Age"] == ">18 to 22", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "adult", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "? & 19", "New Age"] = "19"
sharkattack_main.loc[sharkattack_main["New Age"] == "25 to 35", "New Age"] = "25"
sharkattack_main.loc[sharkattack_main["New Age"] == "23 & 26", "New Age"] = "23"
sharkattack_main.loc[sharkattack_main["New Age"] == "(adult)", "New Age"] = "18"
sharkattack_main.loc[sharkattack_main["New Age"] == "33 & 37", "New Age"] = "33"
sharkattack_main.loc[sharkattack_main["New Age"] == "25 or 28", "New Age"] = "25"
sharkattack_main.loc[sharkattack_main["New Age"] == "37, 67, 35, 27,  ? & 27", "New Age"] = "27"
sharkattack_main.loc[sharkattack_main["New Age"] == "21, 34,24 & 35", "New Age"] = "21"
sharkattack_main.loc[sharkattack_main["New Age"] == "30 & 32", "New Age"] = "20"
sharkattack_main.loc[sharkattack_main["New Age"] == "50 & 30", "New Age"] = "30"
sharkattack_main.loc[sharkattack_main["New Age"] == '"middle-age"', "New Age"] = "50"
sharkattack_main.loc[sharkattack_main["New Age"] == '34 & 19', "New Age"] = "50"
sharkattack_main.loc[sharkattack_main["New Age"] == '33 & 26', "New Age"] = "26"
sharkattack_main.loc[sharkattack_main["New Age"] == '36 & 23', "New Age"] = "23"
sharkattack_main.loc[sharkattack_main["New Age"] == '31 or 33', "New Age"] = "31"

In [93]:
#COLUNA AGE - Limpando as strings (part 1: transformando string in numérico para criar categoria)
sharkattack_main['New Age'] = sharkattack_main['New Age'].str.strip()

In [94]:
#COLUNA AGE - Limpando as strings estranhas (part 13: transformando valores NaN)

sharkattack_main.loc[sharkattack_main["New Age"] == '!!', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'M', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'X', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'F', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'A.M.', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'Ca. 33', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == '', "New Age"] = np.nan
sharkattack_main.loc[sharkattack_main["New Age"] == 'MAKE LINE GREEN', "New Age"] = np.nan

In [95]:
sharkattack_main['New Age'].unique()

array(['67', '30', '60', '10', nan, '15', '14', '6', '32', '59', '41',
       '16', '8', '49', '34', '53', '26', '68', '43', '50', '29', '40',
       '51', '19', '38', '17', '31', '39', '58', '13', '33', '28', '35',
       '62', '20', '22', '56', '12', '21', '42', '36', '18', '37', '9',
       '24', '11', '27', '57', '25', '64', '47', '55', '7', '71', '48',
       '54', '75', '46', '61', '73', '52', '70', '23', '4', '63', '45',
       '44', '65', '74', '69', '3', '82', '66', '72', '1', '77', '5',
       '86', '84', '87', '81', '78'], dtype=object)

In [96]:
#COLUNA AGE - Criando coluna de categoria por idade (part 1)
sharkattack_main['New Age'].fillna("-1", inplace=True)

In [97]:
sharkattack_main['New Age'] = sharkattack_main['New Age'].astype(int)

def categoria_idade(row):
  if row >= 18:
    return "Adult"
  elif row < 18 and row > 0:
    return "Child"
  else: 
    return np.nan

sharkattack_main['Category_Age'] = sharkattack_main['New Age'].apply(categoria_idade)

In [98]:
#COLUNA AGE - Reorganizando colunas
sharkattack_main = sharkattack_main.loc[:,['index', 'Date', 'Year', 'Type', 'Country', 'Update Countries', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'New Age', 'Category_Age', 'Injury',
       'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source']]


In [99]:
sharkattack_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6811 entries, 0 to 6810
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   6811 non-null   int64 
 1   Date                    6811 non-null   object
 2   Year                    6809 non-null   object
 3   Type                    6791 non-null   object
 4   Country                 6792 non-null   object
 5   Update Countries        6811 non-null   object
 6   Area                    6362 non-null   object
 7   Location                6279 non-null   object
 8   Activity                6235 non-null   object
 9   Name                    6592 non-null   object
 10  Sex                     6236 non-null   object
 11  Age                     3863 non-null   object
 12  New Age                 6811 non-null   int64 
 13  Category_Age            3873 non-null   object
 14  Injury                  6778 non-null   object
 15  Fata

#Coluna Sex

In [100]:
#COLUNA SEX - Investigando valores (part 1: Descobrindo valores na coluna)
sharkattack_main['Sex '].value_counts()

M        5481
F         748
M           2
 M          1
lli         1
M x 2       1
N           1
.           1
Name: Sex , dtype: int64

In [101]:
#COLUNA SEX - Alterando valores possíveis (part 1: Alterando valores)
sharkattack_main['Sex '] = sharkattack_main['Sex '].str.strip()
sharkattack_main.loc[sharkattack_main["Sex "] == "lli", "Sex "] = "M"
sharkattack_main['Sex '] = sharkattack_main['Sex '].replace(".", np.nan)
sharkattack_main.loc[sharkattack_main["Sex "] == "M x 2", "Sex "] = "M"
sharkattack_main.loc[sharkattack_main["Sex "] == "N", "Sex "] = "M"

sharkattack_main['Sex '].value_counts()

M    5487
F     748
Name: Sex , dtype: int64

In [102]:
#COLUNA SEX - Alterando valores possíveis (part 2: Alterando valores Male/Men/Man)
regex_male = r'\bman\b|\bmen\b|\bmale\b|\bmales\b'
sharkattack_main.loc[(sharkattack_main['Sex '].isnull()) & (sharkattack_main['Name'].str.contains(regex_male, case=False, na=False)), "Sex "] = "M"


In [103]:
#COLUNA SEX - Alterando valores possíveis (part 2.1: Alterando valores Female/Woman/Women)
regex_female = r'\bwoman\b|\bwomen\b|\bfemale\b|\bfemales\b'
sharkattack_main.loc[(sharkattack_main['Sex '].isnull()) & (sharkattack_main['Name'].str.contains(regex_female, case=False, na=False)), "Sex "] = "F"


In [104]:
sharkattack_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6811 entries, 0 to 6810
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   6811 non-null   int64 
 1   Date                    6811 non-null   object
 2   Year                    6809 non-null   object
 3   Type                    6791 non-null   object
 4   Country                 6792 non-null   object
 5   Update Countries        6811 non-null   object
 6   Area                    6362 non-null   object
 7   Location                6279 non-null   object
 8   Activity                6235 non-null   object
 9   Name                    6592 non-null   object
 10  Sex                     6256 non-null   object
 11  Age                     3863 non-null   object
 12  New Age                 6811 non-null   int64 
 13  Category_Age            3873 non-null   object
 14  Injury                  6778 non-null   object
 15  Fata

In [105]:
#COLUNA SEX - Visualizando os NaN (part 1: Visualizando NaN para ter certeza que não é possível alterar)
sharkattack_main.loc[(sharkattack_main['Sex '].isnull()) & (sharkattack_main['Name'].isnull())]


,index,Date,Year,Type,Country,Update Countries,Area,Location,Activity,Name,Sex,Age,New Age,Category_Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
54,54,19-Jul-2022,2022,Unprovoked,USA,USA,Florida,"Cape Canaveral, Brevard County",NaN,NaN,NaN,NaN,-1,NaN,minor injuries,N,10jh45,Shark involvement not confirmed,"WESH, 7/20/2022"
78,78,27-May-2022,2022,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Yallingup,Surfing,NaN,NaN,NaN,-1,NaN,Knocked off board by a shark. No injury,N,NaN,"Bronze whaler shark, 2m","Perth Now, 5/27/2022"
82,82,07-May-2022,2022,Unprovoked,FRENCH POLYNESIA,FRENCH POLYNESIA,Bora Bora,To'opua,NaN,NaN,NaN,NaN,-1,NaN,Hand bitten,N,NaN,NaN,"Franceinfo, 5/10/202232/2021"
215,215,22-Jan-2021,2021,Watercraft,USA,USA,Florida,Off Tampa Bay,Fishing,NaN,NaN,NaN,-1,NaN,No injury to occupants,N,NaN,White shark,"J. McCallister, GSAF"
289,289,20-Jun-2020,2020,Unprovoked,BAHAMAS,BAHAMAS,Exumas,"Pig Beach, Pig Island",NaN,NaN,NaN,NaN,-1,NaN,NaN,N,NaN,NaN,"M. Michaelson, GSAF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6664,6689,1733,1733,Invalid,ICELAND,ICELAND,Bardestrand,Talkknefiord,NaN,NaN,NaN,NaN,-1,NaN,"Partial hominid remains recovered from shark, ...",NaN,NaN,Shark involvement prior to death unconfirmed,E. Olafsen
6697,6724,Before 1934,0000,Unprovoked,URUGUAY,URUGUAY,Rocha,"Isla Chica, La Paloma",Swimming,NaN,NaN,NaN,-1,NaN,Foot bitten,N,NaN,NaN,"Di Candia, 2004"
6708,6735,Before 2012,0000,Unprovoked,SPAIN,SPAIN,Canary Islands,Tenerife,Skin diving,NaN,NaN,NaN,-1,NaN,Injury required 16 stitches,N,NaN,NaN,"C. Moore, GSAF"
6747,6775,"No date, Before 1963",0000,Unprovoked,SINGAPORE,SINGAPORE,NaN,"Keppel Harbor, 2 miles from Singapore city ce...",Swimming,NaN,NaN,NaN,-1,NaN,Recovered,N,NaN,NaN,"V.M. Coppleson (1958), p.266"


In [106]:
#COLUNA SEX - Visualizando os NaN (part 2: Visualizando semi NaN para ter certeza que não é possível alterar)
sharkattack_main.loc[(sharkattack_main['Sex '].isnull()) & (~sharkattack_main['Name'].isnull())]

,index,Date,Year,Type,Country,Update Countries,Area,Location,Activity,Name,Sex,Age,New Age,Category_Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
194,194,17-Apr-2021,2021,Provoked,USA,USA,North Carolina,Outer Banks,Tagging sharks,fishing boat Sarah Brent,NaN,NaN,-1,NaN,"No injury to occupants, shark bit boat after b...",N,NaN,"Mako shark, 4 juvenile","News Observer, 4/29/2021"
259,259,30-Aug-2020,2020,Watercraft,AUSTRALIA,AUSTRALIA,New South Wales,Forster,Fishing / Filming sharks feeding on whale carcass,Occupant: Dean Butler,NaN,NaN,-1,NaN,"Shark bit boat, no injury to occupant",N,Afternoon,White shark,"Nine News, 8/30/2020"
274,274,29-Jul-2020,2020,Watercraft,AUSTRALIA,AUSTRALIA,Tasmania,Tenth Island,Sightseeing,5.5 m runabout. Occupants: Sean & James Vinar,NaN,NaN,-1,NaN,"No injury to occupants, injury to shark attemp...",NaN,09h08,"White shark, 4m","C. Black, GSAF"
569,569,Reported 25-Nov-2017,2017,Sea Disaster,LIBYA,LIBYA,NaN,Gars Garabulli,2 boats capsized,31 migrants,NaN,NaN,-1,NaN,FATAL,Y,NaN,Some drowned but other may have been killed by...,"TG Com 24, 11/25/2017"
691,692,01-Feb-2017,2017,Watercraft,USA,USA,South Carolina,16 miles off Hilton Head,Fishing,"Tagging sharks, Chip Michelove & crew",NaN,NaN,-1,NaN,"Shark bit boat, no injury to occupants",N,NaN,"White shark, female, 14'","YouTube, 2/2/2017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,6698,Reported 1637,1637,Unprovoked,INDIA,INDIA,West Bengal,Hooghly River mouth,Wading,Hindu pilgrims,NaN,NaN,-1,NaN,NaN,UNKNOWN,NaN,NaN,"H. Edwards, p.31, citing Sebastian Manrique"
6673,6699,Reported 1617,1617,Unprovoked,INDIA,INDIA,West Bengal,Ganges Delta,NaN,Indian people,NaN,NaN,-1,NaN,NaN,UNKNOWN,NaN,NaN,"H. Edwards, p.31, citing Samuel Purchas"
6712,6740,Before 1963,0000,Unprovoked,DJIBOUTI,DJIBOUTI,Gulf of Tadjoura,NaN,A dhow capsized,Passenger & crew,NaN,NaN,-1,NaN,FATAL,Y,NaN,NaN,A. C. Doyle
6722,6750,Before 2004,0000,Watercraft,MOZAMBIQUE,MOZAMBIQUE,Inhambane Province,Off Inhambane,Fishing,"4.8-metre skiboat, Occupants: Rod Salm & 4 fri...",NaN,NaN,-1,NaN,"No injury to occupants, shark bumped boat",N,NaN,Whale shark,South African Shark Attack File


In [107]:
sharkattack_main['Sex '].value_counts()

M    5506
F     750
Name: Sex , dtype: int64

#Dados para Hipóteses

In [108]:
#Hipóteses para Validar
# Hipótese mãe: Existem diferença entre quantidade de ataques dependendo do sexo, país e idade
##1 Ataques de tubarão acontecem acontecem 2x nos EUA do que eu qualquer outro país do mundo
##2 Ataques de tubarão acontecem mais em homens do que em mulheres 
##3 Ataques de tubarão acontecem mais em adultos (+18 anos) do que em crianças.

In [109]:
#CRIANDO UMA COLUNA DE CONTAGEM
sharkattack_main['Attack Count'] = pd.Series(1, index=sharkattack_main.index)
sharkattack_main

,index,Date,Year,Type,Country,Update Countries,Area,Location,Activity,Name,Sex,Age,New Age,Category_Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Attack Count
0,0,25-Mar-2023,2023,Unprovoked,USA,USA,Florida,"Blue Fish Channel, Key West, Monroe County",Kite-Surfing,Kevin Carlton Scott,M,67,67,Adult,Lacertions to right calf,N,14h00,NaN,"Yahoo News, 3/28/2023",1
1,1,22-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Delta Island,Wading,Adam Norton,M,30s,30,Adult,Lower left leg and foot injured,N,18h00,NaN,"A. Currie, GSAF",1
2,2,19-Mar-2023,2023,Unprovoked,USA,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,60,60,Adult,Left hand and left leg.,N,12h30,NaN,"Khon2, 3/19/2023",1
3,3,17-Mar-2023,2023,Unprovoked,MEXICO,MEXICO,Quintana Roo,Cancún,Swimming,Dillon Armijo,M,10,10,Child,Significant injury to leg,N,NaN,NaN,"Denver7, 3/29/2023",1
4,4,16-Mar-2023,2023,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,NaN,-1,NaN,NaN,N,08h00,5m to 6m shark,"B. Myatt, GSAF",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6806,6835,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,-1,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",1
6807,6836,Before 1903,0000,Unprovoked,AUSTRALIA,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,-1,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",1
6808,6837,1900-1905,0000,Unprovoked,USA,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,-1,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",1
6809,6838,1883-1889,0000,Unprovoked,PANAMA,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,-1,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",1


## Hipótese País

In [110]:
#Número de Ataques total / Número de Países
#Número de Ataques no país comparado com o total de ataques
#Número de Ataques no país comparado com o total do continente que se encontra
#Número de Ataques no país comparado com o total do hemisfério que se encontra

In [111]:
#Número de Ataques no país comparado com o total de ataques

sharkattack_main["Update Countries"].value_counts(normalize=True) * 100

USA                                 36.631919
AUSTRALIA                           21.435912
SOUTH AFRICA                         8.721186
NEW ZEALAND                          2.099545
IN BETWEEN COUNTRIES OR HIGH SEA     1.996770
                                      ...    
NORTHERN MARIANA ISLANDS             0.014682
JAVA                                 0.014682
BAY OF BENGAL                        0.014682
SLOVENIA                             0.014682
CEYLON (SRI LANKA)                   0.014682
Name: Update Countries, Length: 167, dtype: float64

## Hipótese Idade

In [112]:
## Hipótese Idade

In [113]:
sharkattack_main["Category_Age"].value_counts(normalize=True) * 100

Adult    72.166279
Child    27.833721
Name: Category_Age, dtype: float64



##Hipótese Gênero

In [114]:
# Hipótese Gênero

In [115]:
sharkattack_main["Sex "].value_counts(normalize=True) * 100

M    88.011509
F    11.988491
Name: Sex , dtype: float64

#Hipotese Geral

In [145]:
sharkattack_main.groupby("Update Countries")["Category_Age", "Sex "].value_counts()

<ipython-input-145-311aaf38603d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sharkattack_main.groupby("Update Countries")["Category_Age", "Sex "].value_counts()


Update Countries   Category_Age  Sex 
ADMIRALTY ISLANDS  Adult         M         1
AMERICAN SAMOA     Adult         M         3
ARGENTINA          Adult         M         2
ARUBA              Adult         M         1
AUSTRALIA          Adult         M       560
                                        ... 
VIETNAM            Adult         F         3
                                 M         2
                   Child         M         1
WESTERN SAMOA      Adult         M         1
YEMEN              Adult         M         2
Length: 233, dtype: int64

In [156]:
pivot_table = pd.pivot_table(data=sharkattack_main,
                             columns="Update Countries",
                             index=["Category_Age", "Sex "],
                             values="index",
                             aggfunc="count",)

pivot_table.transpose().sort_values(by=('Adult', 'M'),ascending=False).apply(lambda x:100 * x / float(x.sum())).round(2).head(10)



Category_Age      Adult         Child       
Sex                   F      M      F      M
Update Countries                            
USA               55.72  38.78  64.42  49.88
AUSTRALIA         12.90  23.01  14.90  19.91
SOUTH AFRICA       2.93  11.54   2.40   9.12
BAHAMAS            5.28   2.22    NaN   1.07
BRAZIL             0.88   1.89   0.96   2.61
MEXICO             1.47   1.64   0.48   0.47
NEW ZEALAND        1.17   1.56   4.33   1.18
FIJI               0.88   1.19    NaN   0.47
REUNION            0.29   1.11   0.48   0.71
PAPUA NEW GUINEA   0.59   1.07   1.44   1.66